# Previsão de renda

## Etapa 1 CRISP - DM:
### Entendimento do negócio

O propósito central deste projeto é prever a renda dos indivíduos com base em características socioeconômicas e demográficas. Essa previsão é valiosa para aplicações como análise de crédito, segmentação de clientes e definição de estratégias de marketing. 
O projeto direciona o trabalho para a construção de um modelo preditivo que seja preciso e aplicável em cenários reais, começando por entender bem o contexto do negócio, alinhar expectativas com os objetivos estratégicos e definir métricas claras de sucesso para guiar as próximas etapas da análise.

## Etapa 2 Crisp-DM:
### Entendimento dos dados
Nesta etapa, o foco é compreender a base de dados disponível para verificar sua qualidade e potencial de uso no modelo preditivo. Foram analisadas a estrutura, os tipos de variáveis, a presença de valores ausentes, outliers e distribuições estatísticas. Esse entendimento inicial direciona os próximos passos, pois permite identificar quais variáveis são relevantes, quais precisam de transformação ou limpeza, e quais relações podem existir com a renda, preparando o terreno para a modelagem preditiva.

### Dicionário de dados

| Variável              | Descrição                                                                 | Tipo    |
|-----------------------|---------------------------------------------------------------------------|---------|
| data_ref              | Data de referência do registro                                            | object  |
| id_cliente            | Identificador único do cliente                                            | int64   |
| sexo                  | Sexo do cliente (Masculino/Feminino)                                      | object  |
| posse_de_veiculo      | Indica se o cliente possui veículo                                        | bool    |
| posse_de_imovel       | Indica se o cliente possui imóvel                                         | bool    |
| qtd_filhos            | Quantidade de filhos do cliente                                           | int64   |
| tipo_renda            | Tipo de fonte de renda (ex.: assalariado, autônomo, pensionista)          | object  |
| educacao              | Grau de instrução (ex.: fundamental, médio, superior)                     | object  |
| estado_civil          | Estado civil do cliente (ex.: solteiro, casado, viúvo)                    | object  |
| tipo_residencia       | Tipo de residência (ex.: própria, alugada, financiada)                    | object  |
| idade                 | Idade do cliente em anos                                                  | int64   |
| tempo_emprego         | Tempo de emprego em anos (pode conter valores nulos para não empregados)  | float64 |
| qt_pessoas_residencia | Quantidade de pessoas que residem no domicílio                            | float64 |
| renda                 | Renda mensal declarada do cliente (variável alvo)                         | float64 |

#### Carregando os pacotes
É considerado uma boa prática carregar os pacotes que serão utilizados como a primeira coisa do programa.

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
from sklearn.dummy import DummyRegressor

#### Carregando os dados
O comando pd.read_csv é um comando da biblioteca pandas (pd.) e carrega os dados do arquivo csv indicado para um objeto *dataframe* do pandas.

In [2]:
df = pd.read_csv('/kaggle/input/previsao-de-renda-csv/previsao_de_renda.csv')

#### Entendimento dos dados - Univariada

In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,15000.0,7499.500000,4330.271354,0.000000,3749.750000,7499.500000,11249.250000,14999.000000
index,15000.0,8298.753467,4800.904442,0.000000,4111.750000,8330.500000,12469.000000,16649.000000
qtd_filhos,15000.0,0.437267,0.760594,0.000000,0.000000,0.000000,1.000000,14.000000
idade,15000.0,43.841400,11.229170,22.000000,34.000000,43.000000,53.000000,68.000000
tempo_emprego,12466.0,7.750462,6.780216,0.191781,3.016438,6.016438,10.183562,42.906849
qt_pessoas_residencia,15000.0,2.219067,0.922352,1.000000,2.000000,2.000000,3.000000,15.000000
renda,15000.0,4624.632946,4628.571729,159.900000,1945.740000,3278.260000,5642.365000,89918.040000


In [4]:
cat_cols = df.select_dtypes(include=["object", "bool"])
summary_cat = (
    cat_cols.apply(lambda col: "; ".join(
        [f"{idx}: {val:.2%}" for idx, val in col.value_counts(normalize=True, dropna=False).items()]
    ))
    .reset_index()
    .rename(columns={"index": "variavel", 0: "top_categorias"})
)
display(summary_cat)

,variavel,top_categorias
0,data_ref,2015-01-01: 6.67%; 2015-02-01: 6.67%; 2015-03-...
1,sexo,F: 67.78%; M: 32.22%
2,posse_de_veiculo,False: 61.11%; True: 38.89%
3,posse_de_imovel,True: 67.21%; False: 32.79%
4,tipo_renda,Assalariado: 52.17%; Empresário: 22.07%; Pensi...
5,educacao,Secundário: 59.51%; Superior completo: 35.08%;...
6,estado_civil,Casado: 70.33%; Solteiro: 11.78%; União: 7.79%...
7,tipo_residencia,Casa: 90.44%; Com os pais: 4.25%; Governamenta...
8,mau,False: 97.59%; True: 2.41%


Em resumo: o perfil predominante no dataset é de mulheres, casadas, assalariadas, com ensino secundário, vivendo em casa própria, com cerca de 44 anos, poucos filhos, renda média em torno de R$ 4,6 mil e histórico de mau pagamento relativamente baixo (apenas 2,4%).

### Entendimento dos dados - Bivariadas

In [7]:
bivariada_categoria_cols = df.select_dtypes(include=["object","bool"]).columns.drop("mau", errors="ignore")

summary_bi_cat = (
    bivariada_categoria_cols.to_series()
    .apply(lambda c: df.groupby(c)["mau"].mean().round(3).to_dict())
    .reset_index()
    .rename(columns={"index": "variavel", 0: "taxa_mau"})
)
display(summary_bi_cat)


num_cols = df.select_dtypes(include=["int64","float64"]).columns.drop("mau", errors="ignore")

summary_bi_num = df.groupby("mau")[num_cols].describe().T

display(summary_bi_num)

,variavel,taxa_mau
0,data_ref,"{'2015-01-01': 0.021, '2015-02-01': 0.017, '20..."
1,sexo,"{'F': 0.023, 'M': 0.026}"
2,posse_de_veiculo,"{False: 0.026, True: 0.021}"
3,posse_de_imovel,"{False: 0.029, True: 0.022}"
4,tipo_renda,"{'Assalariado': 0.023, 'Bolsista': 0.0, 'Empre..."
5,educacao,"{'Primário': 0.016, 'Pós graduação': 0.0, 'Sec..."
6,estado_civil,"{'Casado': 0.021, 'Separado': 0.026, 'Solteiro..."
7,tipo_residencia,"{'Aluguel': 0.033, 'Casa': 0.024, 'Com os pais..."


mau                                 False         True 
Unnamed: 0            count  14638.000000    362.000000
                      mean    7486.520768   8024.334254
                      std     4327.621483   4410.256172
                      min        0.000000     23.000000
                      25%     3739.250000   4128.500000
                      50%     7483.500000   8261.500000
                      75%    11229.750000  11870.500000
                      max    14999.000000  14960.000000
index                 count  14638.000000    362.000000
                      mean    8200.087717  12288.447514
                      std     4748.913775   5185.271665
                      min        0.000000    284.000000
                      25%     4061.000000   8606.250000
                      50%     8237.500000  14630.500000
                      75%    12309.750000  16532.250000
                      max    16470.000000  16649.000000
qtd_filhos            count  14638.000000    362.000000
                      mean       0.436125      0.483425
                      std        0.759436      0.805797
                      min        0.000000      0.000000
                      25%        0.000000      0.000000
                      50%        0.000000      0.000000
                      75%        1.000000      1.000000
                      max       14.000000      3.000000
idade                 count  14638.000000    362.000000
                      mean      43.860978     43.049724
                      std       11.218604     11.636830
                      min       22.000000     26.000000
                      25%       34.000000     33.000000
                      50%       43.000000     41.000000
                      75%       53.000000     52.000000
                      max       68.000000     67.000000
tempo_emprego         count  12165.000000    301.000000
                      mean       7.801356      5.693542
                      std        6.798021      5.654413
                      min        0.191781      0.254795
                      25%        3.084932      1.698630
                      50%        6.049315      3.797260
                      75%       10.252055      8.200000
                      max       42.906849     35.142466
qt_pessoas_residencia count  14638.000000    362.000000
                      mean       2.219292      2.209945
                      std        0.920568      0.993112
                      min        1.000000      1.000000
                      25%        2.000000      2.000000
                      50%        2.000000      2.000000
                      75%        3.000000      3.000000
                      max       15.000000      5.000000
renda                 count  14638.000000    362.000000
                      mean    4633.059009   4283.912735
                      std     4641.710913   4052.789611
                      min      159.900000    355.200000
                      25%     1946.012500   1941.647500
                      50%     3292.030000   2856.885000
                      75%     5659.280000   4947.870000
                      max    89918.040000  25268.950000

Em resumo: o perfil mais associado à inadimplência é de homens, sem bens (carro/imóvel), empresários/autônomos, com ensino médio ou superior incompleto, vivendo de aluguel, com menor tempo de emprego e menor renda.

## Etapa 3 Crisp-DM: Preparação dos dados
Nessa etapa realizamos tipicamente as seguintes operações com os dados:
    
  - **seleção**: Já temos os dados selecionados adequadamente?
  - **limpeza**: Precisaremos identificar e tratar dados faltantes
  - **construção**: construção de novas variáveis
  - **integração**: Temos apenas uma fonte de dados, não é necessário integração
  - **formatação**: Os dados já se encontram em formatos úteis? 

In [8]:
#Seleção
df = df.drop(['Unnamed: 0', 'data_ref', 'index'], axis=1)
df.columns

Index(['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos',
       'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'mau', 'renda'],
      dtype='object')

In [9]:
#Limpeza
df['tempo_emprego'] = df['tempo_emprego'].fillna(
    df['tempo_emprego'].median()
)

df['qt_pessoas_residencia'] = df['qt_pessoas_residencia'].fillna(
    df['qt_pessoas_residencia'].median()
)

df.isna().sum()

sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
mau                      0
renda                    0
dtype: int64

In [10]:
#Construção

df['densidade_dependentes'] = df['qtd_filhos'] / df['qt_pessoas_residencia']

df['score_patrimonio'] = df[['posse_de_imovel', 'posse_de_veiculo']].sum(axis=1)

df['educa_renda'] = df['educacao'].astype(str) + "_" + df['tipo_renda'].astype(str)

df['estabilidade'] = df['tempo_emprego'] / df['idade']

In [11]:
#Formatação

for col in ['posse_de_veiculo', 'posse_de_imovel', 'mau']:
    if df[col].dtype == bool:
        df[col] = df[col].astype(int)

cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

ohe = OneHotEncoder(sparse_output=False, drop=None, handle_unknown="ignore")

encoded = ohe.fit_transform(df[cat_cols])

cat_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(cat_cols), index=df.index)

df_num = df.drop(columns=cat_cols)
df = pd.concat([df_num, cat_df], axis=1)

df.dtypes

posse_de_veiculo                                      int64
posse_de_imovel                                       int64
qtd_filhos                                            int64
idade                                                 int64
tempo_emprego                                       float64
qt_pessoas_residencia                               float64
mau                                                   int64
renda                                               float64
densidade_dependentes                               float64
score_patrimonio                                      int64
estabilidade                                        float64
sexo_F                                              float64
sexo_M                                              float64
tipo_renda_Assalariado                              float64
tipo_renda_Bolsista                                 float64
tipo_renda_Empresário                               float64
tipo_renda_Pensionista                  

## Etapa 4 Crisp-DM: Modelagem
Nessa etapa que realizaremos a construção do modelo. Os passos típicos são:
- Selecionar a técnica de modelagem
- Desenho do teste
- Avaliação do modelo


Separando em treino e teste.

In [12]:
X = df.drop(columns=["renda"])
y = df["renda"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### Rodando o modelo


In [17]:
modelos = {
    "Linear": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42)
}

resultados = {}

for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    preds = modelo.predict(X_test)
    
    resultados[nome] = {
        "R2": r2_score(y_test, preds),
        "RMSE": mean_squared_error(y_test, preds, squared=False),
        "MAE": mean_absolute_error(y_test, preds),
        "MedAE": median_absolute_error(y_test, preds)
    }

resultados_df = pd.DataFrame(resultados).T
resultados_df

,R2,RMSE,MAE,MedAE
Linear,0.233567,4124.115399,2562.872856,1821.132690
RandomForest,0.317817,3890.848221,2324.600118,1478.497786
GradientBoosting,0.278485,4001.440164,2501.620911,1770.984081


## Etapa 5 Crisp-DM: Avaliação dos resultados


In [18]:
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

importances = pd.Series(rf_model.feature_importances_, index=X_train.columns)
importances = importances.sort_values(ascending=False).head(15)
print(importances)

tempo_emprego                               0.395636
estabilidade                                0.167793
idade                                       0.097600
sexo_F                                      0.050538
score_patrimonio                            0.023754
posse_de_veiculo                            0.022019
qt_pessoas_residencia                       0.018712
sexo_M                                      0.017354
posse_de_imovel                             0.014091
densidade_dependentes                       0.011766
educacao_Superior completo                  0.010262
tipo_renda_Empresário                       0.009865
qtd_filhos                                  0.009857
tipo_renda_Assalariado                      0.009752
educa_renda_Superior completo_Empresário    0.009692
dtype: float64


In [19]:
dummy = DummyRegressor(strategy="mean")
dummy.fit(X_train, y_train)
preds = dummy.predict(X_test)

print("Baseline R²:", r2_score(y_test, preds))

Baseline R²: -5.1728410590312635e-05


O modelo de RandomForest superou o baseline e mostrou que tempo de emprego, estabilidade e idade são os principais fatores que explicam a renda, com variáveis como gênero, patrimônio e educação contribuindo em menor grau. Apesar de útil para estimativas gerais e análises de perfil, o poder explicativo ainda é limitado para previsões individuais exatas.